In [1]:
from lightautoml.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml.reader.seq import IDSInd, TopInd
from lightautoml.reader.base import DictToPandasSeqReader
from lightautoml.tasks import Task
import pandas as pd
import numpy as np
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.boost_cb import BoostCB

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline

from lightautoml.automl.gpu.blend_gpu import WeightedBlenderGPU
from lightautoml.automl.blend import WeightedBlender

import numpy as np

In [2]:
import pandas as pd
import numpy as np

DATA_FOLDER = '../../data/small_new/relational_data'

fulfilment_center_info = pd.read_csv(f'{DATA_FOLDER}/fulfilment_center_info.csv')
meal_info = pd.read_csv(f'{DATA_FOLDER}/meal_info.csv')
train = pd.read_csv(f'{DATA_FOLDER}/train.csv')

val = 0.05


In [3]:
seq_params = {
              'fulfilment_center_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'center_id', 'to_id': 'center_id'},
                          },
              'meal_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'meal_id', 'to_id': 'meal_id'},
                          },
              }
seq_data = {'fulfilment_center_info': fulfilment_center_info[['center_id',
                                                              'city_code',
                                                              'region_code',
                                                              'op_area']],
       'meal_info': meal_info              
       }

X_train = {'plain':train.sample(frac=0.05).reset_index(drop=True) , 
           'seq': seq_data
          }

task = Task('reg', metric='mae')

roles={'target': 'num_orders'}




In [4]:
task = Task('reg', metric='mae')
roles={'target': 'num_orders'}
reader = DictToPandasSeqReader(task=task, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([BoostLGBM(), BoostCB()], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender=WeightedBlender())

In [5]:
%%time

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)


[15:41:42] Feats was rejected during automatic roles guess: []
[15:41:42] Layer 1 train process start. Time left 9999999996.65 secs
[15:41:43] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[15:41:43] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:41:43] Training until validation scores don't improve for 100 rounds
[15:41:44] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:41:44] Training until validation scores don't improve for 100 rounds
[15:41:45] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:41:45] Training until validation scores don't improve for 100 rounds
[15:41:46] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:41:46] Training until validation scores don't improve for 100 rounds
[15:41:46] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:41:46] Training until validation scores don't improve for 100 rounds
[15:41:47] Fitting Lvl_0_Pipe_0_Mod_0_Lig

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:41:49] bestTest = 123.4067387
[15:41:49] bestIteration = 2999
[15:41:49] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[15:41:49] 0:	learn: 222.4239807	test: 217.4933976	best: 217.4933976 (0)	total: 972us	remaining: 2.92s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:41:51] Stopped by overfitting detector  (100 iterations wait)
[15:41:51] bestTest = 122.4245452
[15:41:51] bestIteration = 2680
[15:41:51] Shrink model to first 2681 iterations.
[15:41:51] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[15:41:51] 0:	learn: 219.5626285	test: 225.6020009	best: 225.6020009 (0)	total: 1.01ms	remaining: 3.03s


...

[15:41:53] bestTest = 128.2956673
[15:41:53] bestIteration = 2999
[15:41:53] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[15:41:53] 0:	learn: 223.1274264	test: 215.6599007	best: 215.6599007 (0)	total: 872us	remaining: 2.62s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:41:54] bestTest = 123.3669529
[15:41:54] bestIteration = 2999
[15:41:54] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[15:41:55] 0:	learn: 220.5711404	test: 222.1088483	best: 222.1088483 (0)	total: 1.41ms	remaining: 4.24s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:41:56] bestTest = 124.7995945
[15:41:56] bestIteration = 2999
[15:41:56] Fitting Lvl_0_Pipe_0_Mod_1_CatBoost finished. score = -124.45856549090024
[15:41:56] Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
[15:41:56] Time left 9999999982.79 secs

[15:41:56] Layer 1 training completed.

[15:41:56] Blending: optimization starts with equal weights and score -109.7023811298606
[15:41:56] Blending: iteration 0: score = -104.85544399548105, weights = [0.91741353 0.08258648]
[15:41:56] Blending: iteration 1: score = -104.85544399548105, weights = [0.91741353 0.08258648]
[15:41:56] Blending: no score update. Terminated

CPU times: user 52.3 s, sys: 1.3 s, total: 53.6 s
Wall time: 17.2 s


In [6]:
from lightautoml.dataset.gpu.gpu_dataset import SeqCudfDataset
from lightautoml.dataset.gpu.gpu_dataset import SeqDaskCudfDataset
from lightautoml.reader.gpu.seq_gpu import IDSIndGPU, TopIndGPU
from lightautoml.reader.gpu.seq_reader_gpu import DictToCudfSeqReader
from lightautoml.reader.gpu.seq_reader_gpu import DictToDaskCudfSeqReader

from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCBGPU

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBMultiSeqSimpleFeaturesGPU

import cudf
import dask_cudf
import cupy as cp

In [7]:
task = Task('reg', metric='mae', device='gpu')
reader = DictToCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([BoostXGB(), BoostCBGPU(gpu_ids=[0])], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender=WeightedBlenderGPU())

In [8]:
oof_pred_gpu = automl.fit_predict(X_train, roles=roles, verbose=3)

Feats was rejected during automatic roles guess: []
[15:41:59] Layer 1 train process start. Time left 9999999999.47 secs
[15:41:59] Start fitting Lvl_0_Pipe_0_Mod_0_XGB ...
[15:41:59] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:42:11] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:42:25] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:42:40] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:42:54] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:43:00] Lvl_0_Pipe_0_Mod_0_XGB fitting and predicting completed
[15:43:00] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU ...
[15:43:00] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:43:04] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:43:09] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


...

[15:43:13] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:43:17] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:43:22] Lvl_0_Pipe_0_Mod_1_CatBoostGPU fitting and predicting completed
[15:43:22] Time left 9999999916.48 secs

[15:43:22] Layer 1 training completed.

[15:43:22] Blending: Optimization starts with equal weights and score -108.46382789293622
[15:43:22] Blending, iter 0: score = -103.0803045008062, weights = [0.9068121  0.09318795]
[15:43:22] Blending, iter 1: score = -103.0803045008062, weights = [0.9068121  0.09318795]
[15:43:22] No score update. Terminated


In [9]:
gpu_inf = automl.predict(X_train)

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-

In [10]:
automl.to_cpu()

AutoML
XGB: {'task': <lightautoml.tasks.base.Task object at 0x7f751d8eae20>, 'optimization_search_space': {}, 'freeze_defaults': True, 'default_params': {'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor', 'task': 'train', 'learning_rate': 0.05, 'max_leaves': 128, 'max_depth': 0, 'verbosity': 0, 'reg_alpha': 1, 'reg_lambda': 0.0, 'gamma': 0.0, 'max_bin': 255, 'n_estimators': 3000, 'early_stopping_rounds': 100, 'random_state': 42}, 'models': [<xgboost.core.Booster object at 0x7f751d839610>, <xgboost.core.Booster object at 0x7f771e124190>, <xgboost.core.Booster object at 0x7f751da16640>, <xgboost.core.Booster object at 0x7f751d7ed0d0>, <xgboost.core.Booster object at 0x7f771c371130>], '_features': ['numcount__fulfilment_center_info', 'numcount__meal_info', 'ord__base_price', 'ord__checkout_price', 'center_id', 'emailer_for_promotion', 'homepage_featured', 'id', 'meal_id', 'week'], 'timer': <lightautoml.utils.timer.TaskTimer object at 0x7f751d87d370>, '_nan_rate': None, 'gpu_ids': No

In [11]:
cpu_inf = automl.predict(X_train)

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-

In [12]:
task = Task('reg', metric='mae', device='mgpu')
reader = DictToDaskCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([BoostXGB(), BoostCBGPU(gpu_ids=[0])], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender = WeightedBlenderGPU())

In [13]:
oof_pred_mgpu = automl.fit_predict(X_train, roles=roles, verbose=3)

Feats was rejected during automatic roles guess: []
[15:43:25] Layer 1 train process start. Time left 9999999998.69 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


[15:43:27] Start fitting Lvl_0_Pipe_0_Mod_0_XGB ...
[15:43:27] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:43:39] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:43:53] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:44:09] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:44:22] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[15:44:28] Lvl_0_Pipe_0_Mod_0_XGB fitting and predicting completed
[15:44:28] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU ...
[15:44:28] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:44:32] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


...

[15:44:41] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:44:45] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[15:44:50] Lvl_0_Pipe_0_Mod_1_CatBoostGPU fitting and predicting completed
[15:44:50] Time left 9999999914.55 secs

[15:44:50] Layer 1 training completed.

[15:44:50] Blending: Optimization starts with equal weights and score -108.46382789293622
[15:44:50] Blending, iter 0: score = -103.0803045008062, weights = [0.9068121  0.09318795]
[15:44:50] Blending, iter 1: score = -103.0803045008062, weights = [0.9068121  0.09318795]
[15:44:50] No score update. Terminated


In [14]:
print(cpu_inf.data.T)
print(gpu_inf.data.T)
print(oof_pred_gpu.data.T)
print(oof_pred.data.T)
print(oof_pred_mgpu.data.T)

[[116.87771 293.11795 120.12668 ... 130.96439 356.57184 927.51373]]
[[125.489815 294.3881   120.933975 ... 130.96439  352.12848  966.6916  ]]
[[102.12276 289.3709  109.21083 ... 115.42837 342.24518 575.8248 ]]
[[ 72.123726 276.05576  122.339066 ... 117.198135 354.2322   745.3138  ]]
[[102.12276 289.3709  109.21083 ... 115.42837 342.24518 575.8248 ]]
